<center><h1 style="font-size:40px;">Project 2<br> GANs
</h1></center>

This project will develop an **Generative adversarial network (GAN)**. The task is to have a two networks: generator and discriminator.

The generator tries to fool the discriminator and create realistic images. The discriminators job is to figure out if the image is real or fake. A GAN network is very suitable when there is no clear pattern or we want to "create" new data samples. A good generator can create new and realistic images from images.

All **tasks** include **TODO's** thare are expected to be done before the deadline. The highlighted **Question's** should be answered in the report. Keep the answers separated so it is easy to read for the grading. Some sections include asserts or an expected result to give a and expected results are given. Some sections does not contain any **TODO's** but is good to understand them. 

For the **report** we have prepared an *Report.ipynb* notebook. The report should act as a summary of your findings and motivate your choice of approach. A better motivation show your understanding of the lab. Dont forget to include all **parts** in the report!

This lab logs metrics to **Tensorboard**, for instructions to install check the introduction pdf.

Good luck!

---

Update the network architecture given in  **build_generator**  and  **build_discriminator**  of the class GANModel. Please note that the current image resolution is set to 32x32 (i.e. IMAGE_WIDTH and IMAGE_HEIGHT values) in the file configGAN.py. 
This way initial experiements can run faster. Once you implement the inital version of the network, please set the resolution values back to 128x128. Experimental results should be provided for this high resolution images.  

**Hint:** As a generator model, you can use the segmentation model implemented in lab03. Do not forget to adapt the input and output shapes of the generator model in this case.

# Task

**TODO:** Implement a genenerator architecture

**TODO:** Implement a discriminator architecture

**TODO:** Select a good metric to evaluate the models performance and argue why the metric give a good overview of the performance.

**Question:** Explain in the report what you did and why. Show a pipeline of the code. 

--- 

**Note:** The current image resolution is set to 32x32 (i.e. IMAGE_WIDTH and IMAGE_HEIGHT) in config.py. 
This way initial experiements can run faster. Once you implement the inital version of the network, please set the resolution values back to 128x128. Experimental results should be provided for this high resolution images.  

**Hint:** As a generator model, you can use the segmentation model implemented in lab03. Do not forget to adapt the input and output shapes of the generator model in this case.



# Import packages

In [1]:
%load_ext autoreload
%autoreload 2
# Hacky solution to ac>cess the global utils package
import sys,os
sys.path.append(os.path.dirname(os.path.realpath('../..')))

In [2]:
from config import LabConfig
import torchmetrics
import transforms as T
import torchvision
import torch
from torch.utils.data import DataLoader
from DL_labs.utils.dataset import LastFramePredictorDataset

## Load config
Note that this lab does not support for fineGrained dataset.

In [3]:
cfg = LabConfig()
cfg.todict()

{'CLASSES': ['square', 'triangle', 'circular', 'background'],
 'fineGrained': False,
 'NUM_CLASSES': 4,
 'training_img_dir': '../../data/FlyingObjectDataset_10K/training',
 'validation_img_dir': '../../data/FlyingObjectDataset_10K/validation',
 'testing_img_dir': '../../data/FlyingObjectDataset_10K/testing',
 'spiral_path': '../data/lab2/spiral.dat',
 'SEED': 420,
 'GPU': -1,
 'IMAGE_WIDTH': 128,
 'IMAGE_HEIGHT': 128,
 'IMAGE_CHANNEL': 3,
 'NUM_WORKERS': 4,
 'BATCH_SIZE': 32,
 'SAVE_EVERY': 1,
 'TENSORBORD_DIR': 'logs/'}

# Dataset
First load the dataloaders for three datasets; train, validation and test. Feel free to test different augmentations, more can be found at *https://pytorch.org/vision/stable/transforms.html*

Note that ToTensor and Rezise are required to reshape and transform the images correct. We do not want to apply augmentation to the test_transform that are applied on the validation and test dataloader. For semantic segmentation we do not only have an input image $x$ but also an target label $y$. If we apply example: *RandomHorizontalFlip* the expected target image will be observed but will not be same rotation.

## Augmentation
To ensure that each image have the same shape we pad the dataset. It is possible to resize the images but then the bounding boxes also have to be transformed to correct shape which is not covered in this project! 

In [31]:
train_transform = T.Compose([
    T.ToTensor(),
    T.Resize((cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH),interpolation=torchvision.transforms.InterpolationMode.NEAREST), 
])
test_transform = T.Compose([
    T.ToTensor(),
    T.Resize((cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH),interpolation=torchvision.transforms.InterpolationMode.NEAREST), #, 
])

## Dataloader

In [43]:
train_dataloader = DataLoader(
    LastFramePredictorDataset(
        cfg.training_img_dir, 
        img_shape=(cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH), 
        transforms=train_transform),
    batch_size=cfg.BATCH_SIZE,
    shuffle=True,
    num_workers=0#cfg.NUM_WORKERS
)
valid_dataloader = DataLoader(
    LastFramePredictorDataset(
        cfg.validation_img_dir, 
        img_shape=(cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH), 
        transforms=test_transform),
    batch_size=cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=cfg.NUM_WORKERS
)
test_dataloader = DataLoader(
    LastFramePredictorDataset(
        cfg.testing_img_dir, 
        img_shape=(cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH), 
        transforms=test_transform),
    batch_size=cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=cfg.NUM_WORKERS
)

predict_dataloader = DataLoader(
    LastFramePredictorDataset(
        cfg.testing_img_dir, 
        img_shape=(cfg.IMAGE_HEIGHT, cfg.IMAGE_WIDTH), 
        predict=True, 
        transform=test_transform),
    batch_size=cfg.BATCH_SIZE,
    shuffle=False,
    num_workers=cfg.NUM_WORKERS
)
print("Data batch generators are created!")

Debugging 00
['000001_square_red_000001.png', '000001_square_red_000002.png', '000001_square_red_000003.png', '000001_square_red_000004.png', '000001_square_red_000005.png', '000001_square_red_000006.png', '000001_square_red_000007.png', '000001_square_red_000008.png', '000001_square_red_000009.png', '000001_square_red_000010.png', '000001_square_red_000011.png', '000001_square_red_000012.png', '000001_square_red_000013.png', '000001_square_red_000014.png', '000001_square_red_000015.png', '000001_square_red_000016.png', '000001_square_red_000017.png', '000001_square_red_000018.png', '000001_square_red_000019.png', '000001_square_red_000020.png', '000001_square_red_000021.png', '000001_square_red_000022.png', '000001_square_red_000023.png', '000001_square_red_000024.png', '000001_square_red_000025.png', '000001_square_red_000026.png', '000001_square_red_000027.png', '000001_square_red_000028.png', '000001_square_red_000029.png', '000001_square_red_000030.png', '000001_square_red_000031.

## Visualise data
To get an idea of the dataset we will first plot the data. This is very important, especially if we perform data augmentation. If the implementation is wrong an qualitative estimation will with a great probability tell if something is wrong.

**OPTIONAL:** Add RandomFlip to *train_transform*. What happens to the images below? Observe that we take the transformations from *transforms.py* to perform same transformations on both image and target.

In [42]:
t_x = next(iter(train_dataloader))

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'generator'>

In [26]:
t_x, t_y = next(iter(train_dataloader))
print(f"x {tuple(t_x.shape)} {t_x.dtype} {t_x.min()} {t_x.max()}")
print(f"y {tuple(t_y.shape)} {t_y.dtype} {t_y.min()} {t_y.max()}")
plot.Segmentation.data(t_x[:,:3], t_y, nrows=1)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/stud/f/fremar16/miniconda3/envs/DL-Labs/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/stud/f/fremar16/miniconda3/envs/DL-Labs/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/stud/f/fremar16/miniconda3/envs/DL-Labs/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py", line 86, in default_collate
    raise TypeError(default_collate_err_msg_format.format(elem_type))
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'generator'>


# Implementation

---